In [3]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import numpy as np
import pandas as pd
import time
import re

In [5]:
seed_address_list=[]
seed_page="https://www.immoregion.fr/vente/maison?page="

for i in range(1,50,1):
    current_page=seed_page+str(i)
    page=requests.get(current_page)
    page_content=BeautifulSoup(page.content,"html.parser")
    for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')}):
        if "id-" in link["href"]:
            seed_address_list.append("https://www.immoregion.fr"+link["href"])
    print("{} round".format(i))
seed_address_list=list(np.unique(seed_address_list))


100%|██████████| 49/49 [02:01<00:00,  2.48s/it]


In [381]:
len(seed_address_list)

977

In [383]:
json_list=[]
for link in seed_address_list:
    try:
        page=requests.get(link)
        if page.status_code==200:
            page_content=BeautifulSoup(page.content,"html.parser")

            price=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[0].find_all("div")[0].text
            area=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[3].find_all("div")[0].text
            furnished=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[2].find_all("div")[0].text
            integral_area=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[4].find_all("div")[0].text
            nmbr_of_rooms=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[5].find_all("div")[0].text
            ref=page_content.find_all("h1",class_="KeyInfoBlockStyle__PdpTitle-sc-1o1h56e-2 hWEtva")[0].find_all("span")[0].text
            description=page_content.find_all("div",class_="DescriptionStyle__DescriptionContentDropdown-dqxmgt-3 dyQEIL")[0].find_all("div")[0].text
            link_address=link
            scrapping_time=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            agency=page_content.find_all("h3",class_="FormAgencyInfoStyle__AgencyName-sc-10hhmvq-5 jfZIgK")[0].text
            address=page_content.find_all("div",class_="block-localisation-address")[0].text

            json_data={"Ref_no":ref,"Price":price,"Surface":area,"furnished":furnished,"integral_area":whole_area,"No_room":nmbr_of_rooms,"Description":description,"Link":link_address,"scrapping_time":scrapping_time,"agency":agency,"address":address}
            json_list.append(json_data)
            time.sleep(2)
        else:
            pass
    except:
        pass

In [384]:
pd.DataFrame(json_list)

,Ref_no,Price,Surface,furnished,integral_area,No_room,Description,Link,scrapping_time,agency,address
0,Réf 6772175,66 000 €,Non,26/05/2020,110 m²,300 m²,Ferme à renover. A 10 minutes de l'autoroute ...,https://www.immoregion.fr/vente/maison/aboncou...,2020-05-26 19:53:58,Wagner Immobilier,54115 AboncourtL'adresse exacte de ce bien n'...
1,Réf 6757885,58 000 €,NC,de l'acquéreur,110 m²,"1,74 ares","MAISON. Laforet immobilier Sarrebourg, vous pr...",https://www.immoregion.fr/vente/maison/abresch...,2020-05-26 19:53:58,Laforet Terre Lorraine - Sarrebourg,57560 AbreschvillerL'adresse exacte de ce bie...
2,Réf 5816747,128 800 €,Oui,Immédiate,110 m²,165 m²,"Exclusif Agence !!!, Idéalement situé au cœur ...",https://www.immoregion.fr/vente/maison/algrang...,2020-05-26 19:53:59,Florange Immobilier,57440 AlgrangeL'adresse exacte de ce bien n'e...
3,Réf 6552430,139 000 €,Détails,Non,110 m²,"1,82 ares",Maison à rénover à Algrange Règles sanitaires ...,https://www.immoregion.fr/vente/maison/algrang...,2020-05-26 19:53:59,SC Borbiconi Immobilier,57440 AlgrangeL'adresse exacte de ce bien n'e...
4,Réf 6733873,234 625 €,Non,30/04/2020,110 m²,100 m²,BAHAMAS SUITE PARENTALE - ALSTING. Construise...,https://www.immoregion.fr/vente/maison/alsting...,2020-05-26 19:54:00,Maisons Vesta,57515 AlstingL'adresse exacte de ce bien n'es...
...,...,...,...,...,...,...,...,...,...,...,...
967,Réf 6354419,335 000 €,Non,10/07/2019,110 m²,"96,82 m²",ZOUFFTGEN - Projet COCCINELLE - Lot no6 - Mais...,https://www.immoregion.fr/vente/maison/zoufftg...,2020-05-26 20:07:02,Open Immobilier,57330 ZoufftgenL'adresse exacte de ce bien n'...
968,Réf 6354426,350 000 €,Non,10/07/2019,110 m²,"112,08 m²",ZOUFFTGEN - Projet COCCINELLE - Lot no13 - Mai...,https://www.immoregion.fr/vente/maison/zoufftg...,2020-05-26 20:07:02,Open Immobilier,57330 ZoufftgenL'adresse exacte de ce bien n'...
969,Réf 6703260,223 000 €,Non,26/05/2020,110 m²,130 m²,MAISON 5 PIECES 130 M2. ZOUFFTGEN - A seuleme...,https://www.immoregion.fr/vente/maison/zoufftg...,2020-05-26 20:07:03,Open Immobilier,57330 ZoufftgenL'adresse exacte de ce bien n'...
970,Réf 6728868,330 000 €,Non,26/05/2020,110 m²,"96,82 m²",ZOUFFTGEN - Projet COCCINELLE - Lot no2 - Mais...,https://www.immoregion.fr/vente/maison/zoufftg...,2020-05-26 20:07:04,Open Immobilier,57330 ZoufftgenL'adresse exacte de ce bien n'...


In [369]:
#get new links which are similar to the one you have looked at

seed_address="https://www.immoregion.fr/vente/maison/metz/id-6708871.html"
seed_address_list=[]

for i in range(0,3,1):
    if len(seed_address_list)==0:
        #add original seed_address
        print("seed_address_list is empty")
        seed_address_list.append(seed_address) 
        #read the original seed address and if reading process successful load html in var pagecontent
        page=requests.get(seed_address)                 
        if page.status_code==200:
            print("seed page loaded")
            page_content=BeautifulSoup(page.content,"html.parser")
            #iterate over 10 found links to other offers
            for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')})[-3:]:
                new_link="https://www.immoregion.fr"+link["href"]
                seed_address_list.append(new_link)
                print("link from seed_page added")
                print(new_link)
            counter=1
        else:
            counter=1
        print("erste Schleife zuende")
        print(seed_address_list)
        print("\n")    
    else:
        print("seed_address not empty")
        print("counter: {}".format(counter))
        for link in seed_address_list[counter:]:
            page=requests.get(link)
            if page.status_code==200:
                page_content=BeautifulSoup(page.content,"html.parser")
                #iterate over n found links to other offers
                print("Länge Link Liste: {}".format(len(page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')}))))
                for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')})[-3:]:
                    new_link="https://www.immoregion.fr"+link["href"]
                    new_links.append(new_link)
                    print("link from seed_page added")
                    print(new_link)

            else:
                pass
        print("\n")
        [seed_address_list.append(address) for address in new_links]
        print("seed_address_list updated")
        print("Länge Seed address_list: {}".format(len(seed_address_list)))
        counter+=3
        print("Counter updated: {}".format(counter))
        new_links=[]
        print("\n")